In [1]:
# Enable import from parent package
import sys
import os

sys.path.append("/Users/hyunsukim/Documents/GitHub/Paper-Study/Implicit_Neural_Representations/SIREN/siren")

from siren import dataio, meta_modules, utils, training, loss_functions, modules

from torch.utils.data import DataLoader
import configargparse
from functools import partial
import skvideo.datasets
from siren import dataio
import skvideo.datasets

/Users/hyunsukim/Documents/GitHub/Paper-Study/Implicit_Neural_Representations/SIREN/siren/training.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
vid_dataset = dataio.Video(skvideo.datasets.bikes())
coord_dataset = dataio.Implicit3DWrapper(vid_dataset, sidelength=vid_dataset.shape, sample_fraction=38e-4)
dataloader = DataLoader(coord_dataset, shuffle=True, batch_size=32, pin_memory=True, num_workers=0)



In [5]:
coord_dataset.N_samples, coord_dataset.mgrid.shape

(165376, torch.Size([43520000, 3]))

In [ ]:
coords, img = batch[0]['coords'], batch[1]['img']
coords.shape, img.shape

In [ ]:
vid_dataset[1].shape

In [ ]:
165376 / 640 / 272

In [ ]:
3 * (3,)

In [6]:
import torch 
import numpy as np
def get_mgrid(sidelen, dim=2):
    '''Generates a flattened grid of (x,y,...) coordinates in a range of -1 to 1.'''
    if isinstance(sidelen, int):
        sidelen = dim * (sidelen,)

    if dim == 2:
        pixel_coords = np.stack(np.mgrid[:sidelen[0], :sidelen[1]], axis=-1)[None, ...].astype(np.float32)
        pixel_coords[0, :, :, 0] = pixel_coords[0, :, :, 0] / (sidelen[0] - 1)
        pixel_coords[0, :, :, 1] = pixel_coords[0, :, :, 1] / (sidelen[1] - 1)
    elif dim == 3:
        pixel_coords = np.stack(np.mgrid[:sidelen[0], :sidelen[1], :sidelen[2]], axis=-1)[None, ...].astype(np.float32)
        pixel_coords[..., 0] = pixel_coords[..., 0] / max(sidelen[0] - 1, 1)
        pixel_coords[..., 1] = pixel_coords[..., 1] / (sidelen[1] - 1)
        pixel_coords[..., 2] = pixel_coords[..., 2] / (sidelen[2] - 1)
    else:
        raise NotImplementedError('Not implemented for dim=%d' % dim)

    pixel_coords -= 0.5
    pixel_coords *= 2.
    pixel_coords = torch.Tensor(pixel_coords).view(-1, dim)
    return pixel_coords

In [9]:
vid_dataset.shape

(250, 272, 640)

In [14]:
get_mgrid(vid_dataset.shape, dim=3).shape

torch.Size([43520000, 3])

In [8]:
68000 

17000000

In [10]:
640 * 272 * 250

43520000

In [18]:
np.mgrid[:3,:5,:6]

array([[[[0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0]],

        [[1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1]],

        [[2, 2, 2, 2, 2, 2],
         [2, 2, 2, 2, 2, 2],
         [2, 2, 2, 2, 2, 2],
         [2, 2, 2, 2, 2, 2],
         [2, 2, 2, 2, 2, 2]]],


       [[[0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1],
         [2, 2, 2, 2, 2, 2],
         [3, 3, 3, 3, 3, 3],
         [4, 4, 4, 4, 4, 4]],

        [[0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1],
         [2, 2, 2, 2, 2, 2],
         [3, 3, 3, 3, 3, 3],
         [4, 4, 4, 4, 4, 4]],

        [[0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1],
         [2, 2, 2, 2, 2, 2],
         [3, 3, 3, 3, 3, 3],
         [4, 4, 4, 4, 4, 4]]],


       [[[0, 1, 2, 3, 4, 5],
         [0, 1, 2, 3, 4, 5],
         [0, 1, 2, 3, 4, 5],
         [0, 1, 2, 3, 4, 5]